In [28]:
import pandas as pd

# tira as mensagens de warning
pd.options.mode.chained_assignment = None
# pega os dados
df = pd.read_csv('data/student_records.csv')
df

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Henry,A,Y,90,85,Yes
1,John,C,N,85,51,Yes
2,David,F,N,10,17,No
3,Holmes,B,Y,75,71,No
4,Marvin,E,N,20,30,No
5,Simon,A,Y,92,79,Yes
6,Robert,B,Y,60,59,No
7,Trent,C,Y,75,33,No


In [29]:
# pegar as labels de treinamento e de predição
feature_names = ['OverallGrade', 'Obedient', 'ResearchScore', 'ProjectScore']
training_features = df[feature_names]

outcome_name = ['Recommend']
outcome_labels = df[outcome_name]

In [30]:
# features de treinamento
training_features

,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,90,85
1,C,N,85,51
2,F,N,10,17
3,B,Y,75,71
4,E,N,20,30
5,A,Y,92,79
6,B,Y,60,59
7,C,Y,75,33


In [31]:
# labels de predição
outcome_labels

,Recommend
0,Yes
1,Yes
2,No
3,No
4,No
5,Yes
6,No
7,No


In [32]:
# separar dados numéricos e categóricos
numeric_feature_names = ['ResearchScore', 'ProjectScore']
categoricial_feature_names = ['OverallGrade', 'Obedient']

In [33]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

# faz o fit nos dados numéricos
ss.fit(training_features[numeric_feature_names])

# dimensiona atributos numéricos
training_features[numeric_feature_names] = ss.transform(training_features[numeric_feature_names])

# view updated featureset
training_features

,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,0.899583,1.376650
1,C,N,0.730648,-0.091777
2,F,N,-1.803390,-1.560203
3,B,Y,0.392776,0.772004
4,E,N,-1.465519,-0.998746
5,A,Y,0.967158,1.117516
6,B,Y,-0.114032,0.253735
7,C,Y,0.392776,-0.869179


In [34]:
training_features = pd.get_dummies(training_features, columns=categoricial_feature_names)

# ver os atributos categóricos como "numéricos"
training_features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_B,OverallGrade_C,OverallGrade_E,OverallGrade_F,Obedient_N,Obedient_Y
0,0.899583,1.376650,1,0,0,0,0,0,1
1,0.730648,-0.091777,0,0,1,0,0,1,0
2,-1.803390,-1.560203,0,0,0,0,1,1,0
3,0.392776,0.772004,0,1,0,0,0,0,1
4,-1.465519,-0.998746,0,0,0,1,0,1,0
5,0.967158,1.117516,1,0,0,0,0,0,1
6,-0.114032,0.253735,0,1,0,0,0,0,1
7,0.392776,-0.869179,0,0,1,0,0,0,1


In [35]:
# pegar a lista de atributos categóricos convertidos
categorical_engineered_features = list(set(training_features.columns) - set(numeric_feature_names))

categorical_engineered_features

['Obedient_N',
 'OverallGrade_F',
 'OverallGrade_C',
 'OverallGrade_E',
 'Obedient_Y',
 'OverallGrade_B',
 'OverallGrade_A']

In [36]:
from sklearn.linear_model import LogisticRegression
import numpy as np

# dar o fit no modelo
lr = LogisticRegression()
model = lr.fit(training_features, np.array(outcome_labels['Recommend']))

# view model parameters
model

LogisticRegression()

In [37]:
# avaliação simples do modelo preditivo
pred_labels = model.predict(training_features)
actual_labels = np.array(outcome_labels['Recommend'])

# avaliar a performance do modelo
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print('Accuracy:', float(accuracy_score(actual_labels, pred_labels))*100, '%') 
print('Classification Stats:')
print(classification_report(actual_labels, pred_labels))

Accuracy: 100.0 %
Classification Stats:
              precision    recall  f1-score   support

          No       1.00      1.00      1.00         5
         Yes       1.00      1.00      1.00         3

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



In [38]:
import joblib
import os

# cria a pasta no SO se ela não existir
if not os.path.exists('Model'):
    os.mkdir('Model')
if not os.path.exists('Scaler'):
    os.mkdir('Scaler')

joblib.dump(model, r'Model/model.pickle')
joblib.dump(ss, r'Scaler/scaler.pickle')

['Scaler/scaler.pickle']

In [39]:
# carregar o modelo e o scaler
model = joblib.load(r'Model/model.pickle')
scaler = joblib.load(r'Scaler/scaler.pickle')

In [40]:
# cria outros alunos para testar o modelo
new_data = pd.DataFrame([
    {'Name': 'Nathan', 'OverallGrade': 'F', 'Obedient': 'N', 'ResearchScore': 30, 'ProjectScore': 20}, 
    {'Name': 'Thomas', 'OverallGrade': 'A', 'Obedient': 'Y', 'ResearchScore': 78, 'ProjectScore': 80}
])
new_data = new_data[['Name', 'OverallGrade', 'Obedient', 'ResearchScore', 'ProjectScore']]

new_data

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore
0,Nathan,F,N,30,20
1,Thomas,A,Y,78,80


In [41]:
# pega as features dos novos alunos
prediction_features = new_data[feature_names]

# faz o processo de scale
prediction_features[numeric_feature_names] = scaler.transform(prediction_features[numeric_feature_names])

# cuidando dos atributos categóricos
prediction_features = pd.get_dummies(prediction_features, columns=categoricial_feature_names)

# conjunto de features
prediction_features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_F,Obedient_N,Obedient_Y
0,-1.127647,-1.430636,0,1,1,0
1,0.494137,1.160705,1,0,0,1


In [43]:
# pega as colunas que não existem nesse dataset mas foram treinadas pelo modelo
current_categorical_engineered_features = set(prediction_features.columns) - set(numeric_feature_names)
missing_features = set(categorical_engineered_features) - current_categorical_engineered_features

for feature in missing_features:
    # adiciona zero pra todas as colunas não existentes
    prediction_features[feature] = [0] * len(prediction_features)

# conjunto de features atualizado
prediction_features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_F,Obedient_N,Obedient_Y,OverallGrade_C,OverallGrade_B,OverallGrade_E
0,-1.127647,-1.430636,0,1,1,0,0,0,0
1,0.494137,1.160705,1,0,0,1,0,0,0


In [44]:
# realiza a predição
predictions = model.predict(prediction_features)

#anexa na coluna de recomendação os resultados
new_data['Recommend'] = predictions
new_data

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Nathan,F,N,30,20,No
1,Thomas,A,Y,78,80,Yes
